<a href="https://colab.research.google.com/github/pratap977/CS6910/blob/main/Assignment-2/Part-B/Part-B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image as im
import math as mh
from matplotlib import pyplot
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten
from keras.preprocessing.image import load_img as im
from keras.preprocessing.image import save_img as sim
import os
from random import shuffle
import numpy as np
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers,models
import keras
from tensorflow.keras.applications import Xception,InceptionV3,InceptionResNetV2,ResNet50

# keras.backend.set_image_data_format('channels_first')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#gettig labels for different classes and assigning values
class_labels = os.listdir("/content/drive/MyDrive/outputfin/train")
lab={l:i for l,i in zip(class_labels,range(10))}




##############################################################
#function for prepating data takes two parameters
#1.drive_path - path to drive i.e training or validataion or test
#2.want_aug - Takes 'YES' or 'NO' values to gather info wheter to augment data or not
def prepare_data(drive_path,want_aug):
  imgs_tr = []
  truth_tr = []
  #data augmentation rules here
  data_augmentation  = transforms.Compose([
          transforms.CenterCrop(224),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
      ])
  for l in class_labels:
    #### for each folder in the val dir we iterate though all the files 
    if l.startswith("."):
      continue
    imgs_per_class = os.listdir(os.path.join(drive_path, l)) #getting list of file names
    for img in imgs_per_class:
      #iterating to each file in the folder 
      img_path = os.path.join(drive_path, l, img)
      img=Image.open(img_path)
      img = img.resize((224,224))
      if img.mode == 'L':
                  continue
      if(want_aug=='YES'):
        normalized_image = data_augmentation(img)
        imgs_tr.append(np.asarray(normalized_image).transpose(1,2,0))
      else:
        normalized_image = img
        imgs_tr.append(np.asarray(normalized_image))
      truth_tr.append(lab[l])
      
  #contain augmented images(imgs_tr) and labels(truth_tr)
  return np.asarray(imgs_tr) , np.asarray(truth_tr) 



#####################################################################
# Preparing training set (with augmentation), validation set and test test
####################################################################
# Preparing training data with augmentation
# Loading data from directory


#############augmented Training data############ 
x_train,y_train=prepare_data("/content/drive/MyDrive/outputfin/train",'YES')


#############augmented validation data############ 
x_val,y_val=prepare_data("/content/drive/MyDrive/outputfin/val",'YES')

#############augmented test data############
x_test,y_test=prepare_data("/content/drive/MyDrive/inaturalist_12K/val",'YES')




#####################################################################
# Preparing training set (without augmentation), validation set and test test
####################################################################
# Preparing training data withou augmentation
# Loading data from directory
#############augmented Training data############ 
x_train_un,y_train_un=prepare_data("/content/drive/MyDrive/outputfin/train",'NO')


#############augmented validation data############ 
x_val_un,y_val_un=prepare_data("/content/drive/MyDrive/outputfin/val",'NO')

#############augmented test data############
x_test_un,y_test_un=prepare_data("/content/drive/MyDrive/inaturalist_12K/val",'NO')




In [ ]:
#this function take the following parameters
#1.type - the type of the pretrained model to be used
#2.freez- which layers to freez (-1 indicates freeizing all layers, 
# negative number 'n' indicates freezing n-1 layers from the back, and positive number 'p' freezing starting p-1 layers)
#3.epochs - number of epochs to fit the model
#4.batch - batchsize for training

#this creates , trains and returns the model 
def finetune(type,freez,epoch,optim,batch):
  model=keras.models.Sequential()
  base=Xception(weights='imagenet',input_shape=(299,299,3),include_top=True)
  if(type=='Xception'):
    base=Xception(weights='imagenet',input_shape=(299,299,3),include_top=True)
    model.add(base)
  elif(type=='InceptionV3'):
    base=InceptionV3(weights='imagenet',input_shape=(299,299,3),include_top=True)
    model.add(base)
  elif(type=='ResNet50'):
    #model.add(keras.layers.Dense(224*224*3,activation='softmax'))
   #model.add(keras.layers.Reshape((224,224,3))
    base=ResNet50(weights='imagenet',input_shape=(299,299,3),include_top=False)
    model.add(base)
  elif(type=='InceptionResNetV2'):
    base=InceptionResNetV2(weights='imagenet',input_shape=(299,299,3),include_top=True)
    model.add(base)
    
  model.add(keras.layers.Flatten())
  model.add(keras.layers.Dense(10,activation='softmax'))

  if(freez==-1):
    base.trainable=False
  elif(freez<0):
    for i in range(k,0,1):
      base.layers[i].trainable = False
  elif(freez>0):
    for i in range(0,k,1):
      base.layers[i].trainable = False
  #wandb.init()
  # model.summary()
  model.compile(loss="sparse_categorical_crossentropy", optimizer=optim, metrics=["accuracy"])
  model.fit(x_train, y_train,epochs=epoch, batch_size=batch, validation_data=(x_val,y_val), callbacks=[WandbCallback()])


  
  #tloss,tacc=model.evaluate(x_te)
  #wandb.log({"Test loss":tloss,"Test Accuracy":tacc})
  

  return model

In [ ]:
sweep_configer = {
                  'method': 'grid',
                  'metric':{
                            'name':'accuracy',
                            'goal':'maximize'
                            },
                  'parameters': {
                                'mod'   : {'values': ['Xception','InceptionV3', 'ResNet50', 'InceptionResNetV2']},
                                'freez'   :{'values':[-1,10,5,-10,-6,7,-9,0]},
                                'epochs'    :{'values':[5,10,15,20]},  
                                 'optimizer'               : {'values' :['sgd','adam','rmsprop','rmsprop'] },
                                  'lr'                      : {'values' :[1e-3,1e-2,2e-3] },
                                  'gama'                    : {'values' :[1e-3,2e-3,5e-3] }, 
                                 'batchsize'               : {'values' :[16,32,64,128] },                       
                              }
                }

In [ ]:
#sweep function for hyperparamater tuning
def hypertune():
  wandb.init(config=sweep_configer,magic=True)
  if(wandb.config.optimizer=="sgd"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="adam"):
    optim = tf.keras.optimizers.Adam(learning_rate=wandb.config.lr)
  elif(wandb.config.optimizer=="momentum"):
    optim = tf.keras.optimizers.SGD(learning_rate=wandb.config.lr,momentum=wandb.config.gama)
  elif(wandb.config.optimizer=="rmsprop"):
    optim = tf.keras.optimizers.RMSprop(learning_rate=wandb.config.lr)
  finetune(wandb.config.mod,wandb.config.freez,wandb.config.epochs,optim,wandb.config.batchsize)



In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

wandb: Currently logged in as: pratap49 (use `wandb login --relogin` to force relogin)


True

In [ ]:


sweep_id = wandb.sweep(sweep_configer)
wandb.agent(sweep_id, function=hypertune)

Create sweep with ID: qqnh1etr
Sweep URL: https://wandb.ai/pratap49/uncategorized/sweeps/qqnh1etr


wandb: Agent Starting Run: vsecq06v with config:
wandb: 	batchsize: 16
wandb: 	epochs: 5
wandb: 	freez: -1
wandb: 	gama: 0.001
wandb: 	lr: 0.001
wandb: 	mod: Xception
wandb: 	optimizer: sgd


Epoch 1/5
50/50 [==============================] - 432s 9s/step - loss: 2.3008 - accuracy: 0.1381 - val_loss: 2.2954 - val_accuracy: 0.1900
Epoch 2/5
50/50 [==============================] - 429s 9s/step - loss: 2.2999 - accuracy: 0.1342 - val_loss: 2.2952 - val_accuracy: 0.1900
Epoch 3/5
50/50 [==============================] - 429s 9s/step - loss: 2.3010 - accuracy: 0.1321 - val_loss: 2.2950 - val_accuracy: 0.1900
Epoch 4/5
50/50 [==============================] - 426s 9s/step - loss: 2.2988 - accuracy: 0.1368 - val_loss: 2.2949 - val_accuracy: 0.1950
Epoch 5/5
50/50 [==============================] - 431s 9s/step - loss: 2.2995 - accuracy: 0.1516 - val_loss: 2.2947 - val_accuracy: 0.1950


epoch,4
loss,2.29992
accuracy,0.13875
val_loss,2.29471
val_accuracy,0.195
_runtime,2157
_timestamp,1619003244
_step,4
best_val_loss,2.29471
best_epoch,4


epoch,▁▃▅▆█
loss,█▆▄▃▁
accuracy,▁▅▃▁█
val_loss,█▆▅▃▁
val_accuracy,▁▁▁██
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


...

Epoch 1/5
50/50 [==============================] - 323s 6s/step - loss: 43.6941 - accuracy: 0.0907 - val_loss: 15.1193 - val_accuracy: 0.1000
Epoch 2/5
50/50 [==============================] - 318s 6s/step - loss: 9.6182 - accuracy: 0.1892 - val_loss: 9.5382 - val_accuracy: 0.1750
Epoch 3/5
50/50 [==============================] - 317s 6s/step - loss: 10.1630 - accuracy: 0.1999 - val_loss: 8.4554 - val_accuracy: 0.2500
Epoch 4/5
50/50 [==============================] - 313s 6s/step - loss: 10.1809 - accuracy: 0.2069 - val_loss: 14.5982 - val_accuracy: 0.1850
Epoch 5/5
50/50 [==============================] - 314s 6s/step - loss: 12.2137 - accuracy: 0.2209 - val_loss: 12.0240 - val_accuracy: 0.1300


epoch,4
loss,10.98863
accuracy,0.24
val_loss,12.02398
val_accuracy,0.13
_runtime,1595
_timestamp,1619014179
_step,4
best_val_loss,8.45542
best_epoch,2


epoch,▁▃▅▆█
loss,█▁▁▁▂
accuracy,▁▅▆▇█
val_loss,█▂▁▇▅
val_accuracy,▁▄█▅▂
_runtime,▁▃▅▆█
_timestamp,▁▃▅▆█
_step,▁▃▅▆█


wandb: Agent Starting Run: kevtau82 with config:
wandb: 	batchsize: 16
wandb: 	epochs: 5
wandb: 	freez: -1
wandb: 	gama: 0.001
wandb: 	lr: 0.001
wandb: 	mod: ResNet50
wandb: 	optimizer: rmsprop


Epoch 1/5
50/50 [==============================] - 322s 6s/step - loss: 82.6611 - accuracy: 0.1081 - val_loss: 40.3996 - val_accuracy: 0.1000
Epoch 2/5
44/50 [=========================>....] - ETA: 30s - loss: 47.3874 - accuracy: 0.1301